# Outils-corpus 4
# Graphes

## 1. Implémentation Python

![graph](http://clement-plancq.github.io/outils-corpus/files/graphe.png)

Ce graphe peut être implémenté à l'aide d'un dictionnaire python.  
En clé les nœuds, en valeur les destinations des arêtes.  
Les arêtes seront des tuples `('A', 'B')` par exemple.

Vous trouverez de l'information de première main sur ces deux sites dont je me suis inspiré :
  - [https://www.python.org/doc/essays/graphs/](https://www.python.org/doc/essays/graphs/)
  - [https://www.python-course.eu/graphs_python.php](https://www.python-course.eu/graphs_python.php)

In [90]:
my_graph = {
    'A': ['B', 'C', 'E'], # le sommet 'A' est relié aux sommets 'B', 'C', 'E'
    'B': ['F'],
    'C': ['G','H'],
    'D': ['H'],
    'E': ['J'],
    'F': ['I'],
    'G': ['C'],
    'H': ['D','J'],
    'I': ['J'],
    'J': ['E', 'J'],
    }

In [22]:
my_graph

{'A': ['B', 'C', 'E'],
 'B': ['F'],
 'F': ['I'],
 'I': ['J'],
 'C': ['G', 'H'],
 'H': ['D', 'J'],
 'E': ['J']}

Pour manipuler un graphe il faut pouvoir ajouter un nœud et une arête

In [10]:
def add_node(graph, node):
    if node not in graph:
        graph[node] = []

In [11]:
def add_edge(graph, edge):
    node1, node2 = tuple(edge)
    if node1 in graph:
        graph[node1].append(node2)
    else:
        graph[node1] = [node2]

In [23]:
# hop j'ajoute un nœud
add_node(my_graph, 'K')
# hop j'ajoute une arête
add_edge(my_graph, ('K', 'J'))
my_graph

{'A': ['B', 'C', 'E'],
 'B': ['F'],
 'F': ['I'],
 'I': ['J'],
 'C': ['G', 'H'],
 'H': ['D', 'J'],
 'E': ['J'],
 'K': ['J']}

Il faut également pouvoir obtenir facilement la listes de nœuds et des arêtes.  
Je fais les nœuds, vous faîtes les arêtes :

In [18]:
def get_nodes(graph):
    """
    retourne les nœuds d'un graphe 
    Args:
        graph (dict): le graphe
    Returns:
        list of str
    """
    return list(graph.keys())

In [19]:
def get_edges(graph):
    """
    retourne les arêtes d'un graphe 
    Args:
        graph (dict): le graphe
    Returns:
        list of tuples
    """
    # your code

In [26]:
get_nodes(my_graph)

['A', 'B', 'F', 'I', 'C', 'H', 'E', 'K']

Et enfin il faut une fonction pour trouver les chemins entre deux nœuds

In [81]:
def find_path(graph, start, end, path=None):
    if path == None:
        path = []
    path = path + [start]
    if start == end:
        return path
    if start not in graph:
        return None
    for node in graph[start]:
        if node not in path:
            extended_path = find_path(graph, node, end, path)
            if extended_path: 
                return extended_path
        return None

In [82]:
find_path(my_graph, 'A', 'J')

['A', 'B', 'F', 'I', 'J']

def find_all_paths(graph, start, end, path=[]):
    if path == None:
        path = []
    path = path + [start]
    if start == end:
        return [path]
    if start not in graph:
        return None
    paths = []
    for node in graph[start]:
        if node not in path:
            newpaths = find_all_paths(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

In [86]:
find_all_paths(my_graph, 'A', 'J')

[['A', 'B', 'F', 'I', 'J'], ['A', 'C', 'H', 'J'], ['A', 'E', 'J']]

In [98]:
from collections import deque
def find_shortest_path(graph, start, end):
        dist = {start: [start]}
        q = deque(start)
        while len(q):
            at = q.popleft()
            for next in graph[at]:
                if next not in dist:
                    dist[next] = dist[at] + [next]
                    q.append(next)
        return dist.get(end)

In [99]:
find_shortest_path(my_graph, 'A', 'J')

['A', 'E', 'J']

## 2. Graphes de données langagières – Grew

Ce qui nous intéresse c'est implémenter sous forme de graphe une donnée comme : ![graph](http://clement-plancq.github.io/outils-corpus/files/sequoia-spacy.svg)

Ce qui suit est inspiré de la [doc de Grew](http://grew.fr/tuto/)  
On a toujours un dictionnaire avec en clé l'identifiant du mot et en valeur un tuple contenant la forme et la liste des dépendants du mot.

In [102]:
g = dict()
g['W1'] = ('Le', [])
g['W2'] = ('conseil', [])
g['W3'] = ('municipal', [])
g['W4'] = ('donne', [('nsubj', 'W2'), ('obj', 'W6')])
g['W5'] = ('son', [])
g['W6'] = ('accord', [('det', 'W5')])
g['W2'][1].append(('det', 'W1'))
g['W2'][1].append(('amod', 'W3'))

In [104]:
g['W4']

('donne', [('nsubj', 'W2'), ('obj', 'W6')])

On peut ajouter des traits au nœud plutôt que la forme seule.

In [109]:
g = dict()  
g['W1'] = ({'form': 'Le', 'upos': 'DET'}, [])
g['W2'] = ({'form': 'conseil', 'upos': 'NOUN'}, [])
g['W3'] = ({'form': 'municipal', 'upos': 'ADJ'}, [])
g['W4'] = ({'form': 'donne', 'upos': 'VERB'}, [('nsubj', 'W2'), ('obj', 'W6')])
g['W5'] = ({'form': 'son', 'upos': 'DET'}, [])
g['W6'] = ({'form': 'accord', 'upos': 'NOUN'}, [('det', 'W5')])
g['W2'][1].append(('det', 'W1'))
g['W2'][1].append(('amod', 'W3'))

Le module Grew permet d'écrire un graphe dans une syntaxe simplifiée
```python
g = grew.graph('''graph {                                                                   
    W1 [form = "Le", lemma = "le", upos = DET];
    W2 [form = "conseil", lemma = "conseil", upos = NOUN];
    W3 [form = "municipal", lemma = "municipal", upos = ADJ];
    W4 [form = "donne", lemma = "donner", upos = VERB];
    W5 [form = "son", lemma = "son", upos = DET];
    W6 [form = "accord", lemma = "accord", upos = NOUN];
    W2 -[det]->W1;
    W2 -[amod]->W3;
    W4 -[nsubj]->W2;
    W4 -[obj]->W6;
    W6 -[det]->W5;
    }''')
```

Il permet également d'utiliser des patrons de recherche comme .
```python
grew.search('''pattern {
     GOV [lemma = "donner"];
     DEP[lemma = "accord"];
     GOV -[obj]-> DEP }', g)
```

Grew peut s'utiliser :
  - en ligne de commande sur des fichiers au format CoNLL-U
  - via la [lib python](https://pypi.org/project/Grew/)

Je ne vous demande pas d'installer Grew sur vos machines. Vous utiliserez l'interface web [Grew-match](http://match.grew.fr/).  
Grew est un outil de réécriture de graphes (*Graph rewriting for NLP*), la recherche de patrons n'en est qu'une sous-partie


Aidez de la [documentation sur les motifs](http://grew.fr/pattern/) et du tutoriel de [Grew-match](http://match.grew.fr/?corpus=UD_French-Sequoia@2.5#) pour trouver dans le corpus UD_French-Sequoia@2.5 :
  - les triplets sujet, verbe, objet
  - les phrases avec sujets inversés


## 3. Graphes de données langagières – Spacy


La librairie [spaCy](https://spacy.io/) propose dans sa chaîne de traitement une analyse syntaxique en dépendance.

Le modèle pour le français est appris sur le corpus [Sequoia](http://deep-sequoia.inria.fr/) version UD et [wikiNER](https://figshare.com/articles/Learning_multilingual_named_entity_recognition_from_Wikipedia/5462500)

Nous verrons plus en détail comment utiliser Spacy lors de la prochaine séance.

In [144]:
import spacy
nlp = spacy.load('fr_core_news_md')
doc = nlp("Le conseil municipal donne son accord pour cette procédure.")
for tok in doc:
    print(tok.text, tok.pos_)

Le DET
conseil NOUN
municipal ADJ
donne VERB
son DET
accord NOUN
pour ADP
cette DET
procédure NOUN
. PUNCT


In [116]:
from spacy import displacy
displacy.render(doc, style="dep", jupyter=True, options={'distance':110})

Pour ce qui nous intéresse aujourd'hui vous devez lire la [doc sur l'analyse en dépendance](https://spacy.io/usage/linguistic-features#dependency-parse)

Pour chaque `token` on peut accéder à:
  - sa tête `token.head`
  - le label de la relation `token.dep_`
  - les tokens régis `token.children` (seulement à gauche `token.lefts`, à droite `token.rights`, la séquence complète `token.subtree`)
  - la chaîne de recteurs `token.ancestors`

In [120]:
root = [token for token in doc if token.head == token][0]
subjects = [tok for tok in root.lefts if tok.dep_ == "nsubj"]
subject = subjects[0]

In [119]:
for descendant in subject.subtree:
    print(descendant.text)

Le
conseil
municipal


In [121]:
root = [token for token in doc if token.head == token][0]
objs = [tok for tok in root.rights if tok.dep_ == "obj"]
obj = objs[0]

In [122]:
for descendant in obj.subtree:
    print(descendant.text)

son
accord


Pour la séance prochaine,  extrayez les triplets (sujet, verbe, objet) des phrases suivantes et commentez les éventuelles erreurs ou manques.

    « Le chat mange la souris. »
    « Les enfants n'aiment pas trop les asperges. »
    « Les Français réclament moins d'impôts. »
    « Les acacias donnent un miel ambré, limpide et fluide. »
    « L'équipe fait porter le chapeau à l'arbitrage. »
